In [6]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [7]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [8]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'serviceVersion': '3.28.2025.1',
 'dwellTimeMS': '0',
 'avgWebHostCPUTotalLoad': 'N/A',
 'dataHitType': 'FROM_MEMORY',
 'Source': 'B11B',
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisp

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [9]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [10]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [400]>

In [11]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [12]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character at: line 1 column 293 (char 292)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [13]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"statusCode":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2547\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2168\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 807\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 261"}'

'{"status":200,"serviceVersion":"3.28.2025.1","dwellTimeMS":"0","avgWebHostCPUTotalLoad":"N/A","dataHitType":"FROM_MEMORY","Source":"B11B","result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [14]:
def each_grade(grades):
    results = []
    for grade in grades:
        example_naep_query = (
            'https://www.nationsreportcard.gov/'
            'Dataservice/GetAdhocData.aspx?'
            f'type=data&subject=writing&grade={grade}&'
            'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')
        naep_resp = requests.get(example_naep_query)
        try:
            naep_resp_j = naep_resp.json()
            naep_resp_d = pd.DataFrame(naep_resp_j['result'])
            results.append(naep_resp_d)
        except Exception as e:
            print(e)
    return pd.concat(results)
grades = ['4', '8', '12']
new_data = each_grade(grades)
print(new_data.head())

Invalid control character at: line 1 column 293 (char 292)
   year sample yearSampleLabel  Cohort CohortLabel stattype subject  grade  \
0  2011     R3            2011       2     Grade 8    MN:MN     WRI      8   
1  2011     R3            2011       2     Grade 8    MN:MN     WRI      8   
0  2011     R3            2011       3    Grade 12    MN:MN     WRI     12   
1  2011     R3            2011       3    Grade 12    MN:MN     WRI     12   

   scale jurisdiction       jurisLabel variable variableLabel varValue  \
0  WRIRP           NP  National public   GENDER        Gender        1   
1  WRIRP           NP  National public   GENDER        Gender        2   
0  WRIRP           NP  National public   GENDER        Gender        1   
1  WRIRP           NP  National public   GENDER        Gender        2   

  varValueLabel       value  isStatDisplayable  errorFlag  
0          Male  139.099505                  1          0  
1        Female  158.567105                  1          0  

In [ ]:
# your code here

grades = ['4','8','12']

R = []

for grade in grades: 
    example_naep_query = (
        'https://www.nationsreportcard.gov/'
        'Dataservice/GetAdhocData.aspx?'
        f'type=data&subject=writing&grade={grade}&'
        'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')
    naep_resp = requests.get(example_naep_query)
    
    try: 
        naep_resp_j = naep_resp.json()
        naep_resp_j
        naep_resp_d = pd.DataFrame(naep_resp_j['result'])
        # result = pd.concat([result,naep_resp_d]) 
        R.append(naep_resp_d)
    
    except Exception as e:
        print(e)
        
pd.concat(R)

# 2. Example 2: needs credentials; no wrapper

Create an account here: https://www.yelp.com/developers/v3/manage_app

In [15]:
## get the key
API_KEY = "98fJNZRA7tQ_2qOQ6JYmCHGZAvksxkZWo3opC3hHLfeg-JuQ-1TWm37oSW_Sz1fDmYojUh17vA7lCxzZPyqEEXeLRJumii9bmqwk2YrLGf2ny5p7A04791VQoMwcaHYx"

In [16]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,NH,03755"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


<Response [200]>

In [17]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': 'wyV_NfYn4ZOfp_sHMDPcAw',
 'alias': 'bistro-at-six-hanover',
 'name': 'Bistro at Six',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/i4jvssUsxa79VYVLOD3TmQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/bistro-at-six-hanover?adjust_creative=8mnpNlIBJN_7p-8VrrLD_Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=8mnpNlIBJN_7p-8VrrLD_Q',
 'review_count': 2,
 'categories': [{'alias': 'lounges', 'title': 'Lounges'},
  {'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
  {'alias': 'tradamerican', 'title': 'American'}],
 'rating': 4.0,
 'coordinates': {'latitude': 43.7001146, 'longitude': -72.2877078},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '6 E South St',
  'address2': None,
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['6 E South St', 'Hanover, NH 03755']},
 'phone': '+16036430600',
 'display_phone': '(603) 643-0600',
 'distance': 198.6517

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,business_hours,attributes
0,wyV_NfYn4ZOfp_sHMDPcAw,bistro-at-six-hanover,Bistro at Six,https://s3-media2.fl.yelpcdn.com/bphoto/i4jvss...,False,https://www.yelp.com/biz/bistro-at-six-hanover...,2,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",4.0,"{'latitude': 43.7001146, 'longitude': -72.2877...",[],$$,"{'address1': '6 E South St', 'address2': None,...",+16036430600,(603) 643-0600,198.651788,"[{'open': [{'is_overnight': True, 'start': '00...","{'business_temp_closed': None, 'menu_url': Non..."
1,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,False,https://www.yelp.com/biz/base-camp-cafe-hanove...,256,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.4,"{'latitude': 43.700626, 'longitude': -72.2887803}",[delivery],$$,"{'address1': '3 Lebanon St', 'address2': 'Ste ...",+16036432007,(603) 643-2007,196.139758,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."
2,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,False,https://www.yelp.com/biz/mollys-restaurant-and...,561,"[{'alias': 'tradamerican', 'title': 'American'...",3.9,"{'latitude': 43.701144, 'longitude': -72.2894249}",[delivery],$$,"{'address1': '43 South Main St', 'address2': '...",+16036432570,(603) 643-2570,250.830160,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."
3,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,False,https://www.yelp.com/biz/lous-restaurant-and-b...,430,"[{'alias': 'tradamerican', 'title': 'American'...",4.2,"{'latitude': 43.70143, 'longitude': -72.289001}",[delivery],$$,"{'address1': '30 S Main St', 'address2': '', '...",+16036433321,(603) 643-3321,245.079232,"[{'open': [{'is_overnight': False, 'start': '0...","{'business_temp_closed': None, 'menu_url': 'ht..."
4,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,False,https://www.yelp.com/biz/sawtooth-kitchen-hano...,30,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.2,"{'latitude': 43.70158, 'longitude': -72.289641}",[],NaN,"{'address1': '33 S Main St', 'address2': '', '...",+16036435134,(603) 643-5134,242.607552,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht..."


In [24]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,MA,02339"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()




## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
##yelp_gendf.head()


## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df


<Response [200]>

{'id': 'Ds-01aoJm7fWNwOBEs01ww',
 'alias': 'square-cafe-hanover-hanover-2',
 'name': 'Square Cafe - Hanover',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/g43wl3os83Lr-Zk1vaQBWA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/square-cafe-hanover-hanover-2?adjust_creative=8mnpNlIBJN_7p-8VrrLD_Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=8mnpNlIBJN_7p-8VrrLD_Q',
 'review_count': 76,
 'categories': [{'alias': 'newamerican', 'title': 'New American'},
  {'alias': 'modern_european', 'title': 'Modern European'},
  {'alias': 'cafes', 'title': 'Cafes'}],
 'rating': 4.1,
 'coordinates': {'latitude': 42.112080082862995,
  'longitude': -70.8165192604065},
 'transactions': [],
 'location': {'address1': '283 Columbia Rd',
  'address2': '',
  'address3': None,
  'city': 'Hanover',
  'zip_code': '02339',
  'country': 'US',
  'state': 'MA',
  'display_address': ['283 Columbia Rd', 'Hanover, MA 02339']},
 'phone': '+17819242581',
 'display_phone': '(78

,id,alias,name,image_url,url,phone,display_phone,price
Ds-01aoJm7fWNwOBEs01ww,Ds-01aoJm7fWNwOBEs01ww,square-cafe-hanover-hanover-2,Square Cafe - Hanover,https://s3-media3.fl.yelpcdn.com/bphoto/g43wl3...,https://www.yelp.com/biz/square-cafe-hanover-h...,+17819242581,(781) 924-2581,NaN
INzblIDtRvklxeQ4utv1Zw,INzblIDtRvklxeQ4utv1Zw,alba-on-53-hanover,Alba on 53,https://s3-media4.fl.yelpcdn.com/bphoto/jlq_Nc...,https://www.yelp.com/biz/alba-on-53-hanover?ad...,+17817895527,(781) 789-5527,NaN
Owps-ltpyRwHpmaEGoSVfw,Owps-ltpyRwHpmaEGoSVfw,machim-house-of-thai-hanover,Machim House of Thai,https://s3-media4.fl.yelpcdn.com/bphoto/zK6Vnv...,https://www.yelp.com/biz/machim-house-of-thai-...,+17819150120,(781) 915-0120,NaN
YTwQH7PfpTQxxRp0A9yCmQ,YTwQH7PfpTQxxRp0A9yCmQ,first-watch-hanover,First Watch,https://s3-media2.fl.yelpcdn.com/bphoto/9Am-r6...,https://www.yelp.com/biz/first-watch-hanover?a...,+17818872588,(781) 887-2588,NaN
rOngCjYgdiyR8r7mKNgqvg,rOngCjYgdiyR8r7mKNgqvg,3brothers-grill-bar-and-restaurant-hanover,3Brothers Grill Bar and Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/GkQ6Du...,https://www.yelp.com/biz/3brothers-grill-bar-a...,+17819243079,(781) 924-3079,$$
ZPn5ARwZt0NVj50AhjwYPA,ZPn5ARwZt0NVj50AhjwYPA,scarlet-oak-tavern-hingham,Scarlet Oak Tavern,https://s3-media4.fl.yelpcdn.com/bphoto/_xdLs1...,https://www.yelp.com/biz/scarlet-oak-tavern-hi...,+17817498200,(781) 749-8200,$$$
OwwRMtXX2P4OB7IIMl3pyg,OwwRMtXX2P4OB7IIMl3pyg,scutari-hanover,Scutari,https://s3-media1.fl.yelpcdn.com/bphoto/te542E...,https://www.yelp.com/biz/scutari-hanover?adjus...,+17814218288,(781) 421-8288,NaN
r4nFoQ0S4QciIg56rFE24g,r4nFoQ0S4QciIg56rFE24g,cheever-tavern-norwell,Cheever Tavern,https://s3-media2.fl.yelpcdn.com/bphoto/llt_df...,https://www.yelp.com/biz/cheever-tavern-norwel...,+17816572700,(781) 657-2700,NaN
_1gdg8Sf6-p-5fyCqN21HA,_1gdg8Sf6-p-5fyCqN21HA,osteria-vivo-pembroke,Osteria Vivo,https://s3-media3.fl.yelpcdn.com/bphoto/BG-VOx...,https://www.yelp.com/biz/osteria-vivo-pembroke...,+17819241315,(781) 924-1315,NaN
XFxKcAecK86SZa1nJhicXA,XFxKcAecK86SZa1nJhicXA,mimas-cuban-cafe-a-taste-of-miami-rockland-2,Mima's Cuban Cafe - A Taste of Miami,https://s3-media1.fl.yelpcdn.com/bphoto/RhC1SH...,https://www.yelp.com/biz/mimas-cuban-cafe-a-ta...,+17818787074,(781) 878-7074,NaN


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://docs.developer.yelp.com/reference/v3_business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [ ]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)
